 #  A Whale off the Port(folio)
 ---

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P 500 Index.

In [2]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path

%matplotlib inline

C:\Users\12152\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\12152\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\12152\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:

* `whale_returns.csv`: Contains returns of some famous "whale" investors' portfolios.

* `algo_returns.csv`: Contains returns from the in-house trading algorithms from Harold's company.

* `sp500_history.csv`: Contains historical closing prices of the S&P 500 Index.

## Whale Returns

Read the Whale Portfolio daily returns and clean the data

In [3]:
#Read in the whale returns to a new dataframe with the date as the index, and then sort the index in ascending order

whale_path = Path("Resources/whale_returns.csv")
whale_df = pd.read_csv(whale_path, index_col="Date", infer_datetime_format=True, parse_dates=True)
whale_df.sort_index(inplace=True)

In [4]:
# Count nulls

whale_df.info()
whale_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1060 entries, 2015-03-02 to 2019-05-01
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   SOROS FUND MANAGEMENT LLC    1059 non-null   float64
 1   PAULSON & CO.INC.            1059 non-null   float64
 2   TIGER GLOBAL MANAGEMENT LLC  1059 non-null   float64
 3   BERKSHIRE HATHAWAY INC       1059 non-null   float64
dtypes: float64(4)
memory usage: 41.4 KB


SOROS FUND MANAGEMENT LLC      1
PAULSON & CO.INC.              1
TIGER GLOBAL MANAGEMENT LLC    1
BERKSHIRE HATHAWAY INC         1
dtype: int64

In [5]:
# Drop nulls

whale_df = whale_df.dropna()

## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data

In [6]:
# Reading algorithmic returns

algo_path = Path("Resources/algo_returns.csv")
algo_df = pd.read_csv(algo_path, index_col="Date", infer_datetime_format=True, parse_dates=True)
algo_df.sort_index(inplace=True)

In [7]:
# Count nulls

algo_df.info()
algo_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1241 entries, 2014-05-28 to 2019-05-01
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Algo 1  1241 non-null   float64
 1   Algo 2  1235 non-null   float64
dtypes: float64(2)
memory usage: 29.1 KB


Algo 1    0
Algo 2    6
dtype: int64

In [8]:
# Drop nulls

algo_df = algo_df.dropna()

## S&P 500 Returns

Read the S&P 500 historic closing prices and create a new daily returns DataFrame from the data. 

In [9]:
# Reading S&P 500 Closing Prices

sp500_prices_path = Path("Resources/sp500_history.csv")
sp500_prices_df = pd.read_csv(sp500_prices_path, index_col="Date", infer_datetime_format=True, parse_dates=True)
sp500_prices_df.sort_index(inplace=True)

In [10]:
# Check Data Types

sp500_prices_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1649 entries, 2012-10-01 to 2019-04-23
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Close   1649 non-null   object
dtypes: object(1)
memory usage: 25.8+ KB


In [11]:
#Remove the dollar signs from the closing prices column (warning message is for future Pandas update, but the code still executes as required here)
sp500_prices_df["Close"] = sp500_prices_df["Close"].str.replace("$","")

C:\Users\12152\AppData\Local\Temp/ipykernel_46860/3613562770.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  sp500_prices_df["Close"] = sp500_prices_df["Close"].str.replace("$","")


In [12]:
#Convert the data type for the closing prices to float
sp500_prices_df["Close"] = sp500_prices_df["Close"].astype(float)

In [13]:
#Calculate the S&P 500 daily returns and save in a new dataframe
sp500_returns_df = sp500_prices_df.pct_change()

In [14]:
#Drop the null values in the S&P 500 returns dataframe and save over the dataframe
sp500_returns_df = sp500_returns_df.dropna()

In [15]:
# Rename the "Close" column since it is now the calculated daily return
sp500_returns_df = sp500_returns_df.rename(columns={"Close":"S&P 500"})

## Combine Whale, Algorithmic, and S&P 500 Returns

In [16]:
#Join Whale Returns, Algorithmic Returns, and the S&P 500 Returns into a single dataframe with columns for each portfolio's returns.
combined_returns = pd.concat([whale_df, algo_df, sp500_returns_df], axis="columns", join="inner")

---

# Conduct Quantitative Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance Anlysis

#### Calculate and Plot the daily returns.

In [ ]:
#Plot the daily returns for each portfolio on a combined plot
combined_returns.plot(title = "Daily Portfolio Returns", figsize = (20,10), ylabel="Daily Return", grid=True)

<AxesSubplot:title={'center':'Daily Portfolio Returns'}, xlabel='Date', ylabel='Daily Return'>

#### Calculate and Plot cumulative returns.

In [ ]:
#Calculate the cumulative returns of each portfolio
combined_cumulative_returns = (1 + combined_returns).cumprod()

#Plot the cumulative returns for each portfolio on a combined plot
combined_cumulative_returns.plot(title = "Cumulative Portfolio Returns", figsize = (20,10), ylabel="Cumulative Return", grid=True)

---

## Risk Analysis

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios
4. Determine which portfolios are riskier than the S&P 500
5. Calculate the Annualized Standard Deviation

### Create a box plot for each portfolio


In [17]:
# Box plot to visually show risk

combined_df.plot.box(figsize=(20,8), title='Portfolio Risk')

NameError: name 'combined_df' is not defined

### Calculate Standard Deviations

In [ ]:
# Calculate the daily standard deviations of all portfolios

combined_df.std()

### Determine which portfolios are riskier than the S&P 500

In [ ]:
# Calculate  the daily standard deviation of S&P 500
SandP_risk = combined_df['S&P500'].std()

# Determine which portfolios are riskier than the S&P 500
combined_df.std()>SandP_risk

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized standard deviation (252 trading days)
annualized_standard_deviantion = combined_df.std()*np.sqrt(252)
annualized_standard_deviantion

---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Calculate and plot the rolling standard deviation for all portfolios using a 21-day window
2. Calculate the correlation between each stock to determine which portfolios may mimick the S&P 500
3. Choose one portfolio, then calculate and plot the 60-day rolling beta between it and the S&P 500

### Calculate and plot rolling `std` for all portfolios with 21-day window

In [ ]:
# Calculate the rolling standard deviation for all portfolios using a 21-day window
combined_df.rolling(window=21).std().plot(figsize=(20,8), title='Standard Deviation')
# Plot the rolling standard deviation
annualized_standard_deviantion.rolling(window=21).std().plot(figsize=(20,8))

### Calculate and plot the correlation

In [ ]:
# Calculate the correlation
price_correlation = combined_df.corr()
# Display de correlation matrix
corr = price_correlation.corr()

corr.style.background_gradient(cmap='coolwarm')

### Calculate and Plot Beta for a chosen portfolio and the S&P 500

In [ ]:
# Calculate covariance of a single portfolio   
combined_df = combined_df.dropna()

covariance = combined_df['Algo 1'].rolling(window=60).cov(combined_df['S&P500'])
                                                        
# Calculate variance of S&P 500
variance = combined_df['S&P500'].rolling(window=60).var()

# Computing beta
beta = covariance / variance
beta
# Plot beta trend
beta.plot();

## Rolling Statistics Challenge: Exponentially Weighted Average 

An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the [`ewm`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html) with a 21-day half-life.

In [ ]:
# Use `ewm` to calculate the rolling window
combined_df.ewm(halflife=21).std().plot(figsize=(20,10));


#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ewm.html

---

# Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. After all, if you could invest in one of two portfolios, and each offered the same 10% return, yet one offered lower risk, you'd take that one, right?

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Annualized Sharpe Ratios

rf_rate = 0.3
sharpe_ratios = ((combined_df.mean() * 252) / (combined_df.std() * np.sqrt(252)))
sharpe_ratios

In [ ]:
# Visualize the sharpe ratios as a bar plot

sharpe_ratios.plot(kind="bar", title="Sharpe Ratios");

### Determine whether the algorithmic strategies outperform both the market (S&P 500) and the whales portfolios.

Write your answer here!

In [ ]:
Determine whether the algorithmic strategies outperform both the market (S&P 500) and the whales portfolios.
Algo 1 outperforms all portfolios

---

# Create Custom Portfolio

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P 500. 

1. Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.
2. Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock
3. Join your portfolio returns to the DataFrame that contains all of the portfolio returns
4. Re-run the performance and risk analysis with your portfolio to see how it compares to the others
5. Include correlation analysis to determine which stocks (if any) are correlated

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

For this demo solution, we fetch data from three companies listes in the S&P 500 index.

* `GOOG` - [Google, LLC](https://en.wikipedia.org/wiki/Google)

* `AAPL` - [Apple Inc.](https://en.wikipedia.org/wiki/Apple_Inc.)

* `COST` - [Costco Wholesale Corporation](https://en.wikipedia.org/wiki/Costco)

In [ ]:
# Get stock data

google_csv = Path("goog_historical.csv")
google_historical = pd.read_csv(
    google_csv, index_col="Trade DATE", infer_datetime_format=True, parse_dates=True)
google_historical = google_historical.sort_index()
google_historical.head()

In [ ]:
# Reading data from 1st stock

apple_csv = Path("aapl_historical.csv")
apple_historical = pd.read_csv(
    apple_csv, index_col="Trade DATE", infer_datetime_format=True, parse_dates=True)
apple_historical = apple_historical.sort_index()
apple_historical.head()


In [ ]:
# Reading data from 2nd stock

apple_csv = Path("aapl_historical.csv")
apple_historical = pd.read_csv(
    apple_csv, index_col="Trade DATE", infer_datetime_format=True, parse_dates=True)
apple_historical = apple_historical.sort_index()
apple_historical.head()


In [ ]:
# Reading data from 3rd stock

costco_csv = Path("cost_historical.csv")
costco_historical = pd.read_csv(
    costco_csv, index_col="Trade DATE", infer_datetime_format=True, parse_dates=True)
costco_historical = costco_historical.sort_index()
costco_historical.head()

In [ ]:
# Combine all stocks in a single DataFrame
stock_data_concatenated = pd.concat([google_historical, apple_historical, costco_historical], axis="rows", join="inner")
stock_data_concatenated.head()


In [ ]:
# Reset Date index

stock_data_concatenated = stock_data_concatenated.reset_index()
stock_data_concatenated.head()


In [ ]:
# Reorganize portfolio data by having a column per symbol

reorganized_portfolio = stock_data_concatenated.pivot_table(values="NOCP", index="Trade DATE", columns="Symbol")
reorganized_portfolio.head()

In [ ]:
# Calculate daily returns

daily_returns = reorganized_portfolio.pct_change()

# Drop NAs
daily_returns = daily_returns.dropna()

# Display sample data
daily_returns.head()

## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [ ]:
# Set weights
weights = [1/3, 1/3, 1/3]

# Calculate portfolio return
weighted_returns = daily_returns.dot(weights)

# Display sample data
weighted_returns.head()

## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [ ]:
# Join your returns DataFrame to the original returns DataFrame
combined_df["Customized"] = weighted_returns
combined_df.tail()


In [ ]:
# Only compare dates where return data exists for all the stocks (drop NaNs)
combined_df = combined_df.dropna()

## Re-run the risk analysis with your portfolio to see how it compares to the others

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized `std`
annualized_std = combined_df.std() * np.sqrt(252)
annualized_std


### Calculate and plot rolling `std` with 21-day window

In [ ]:
# Calculate rolling standard deviation
rolling_std = combined_df.rolling(window=21).std()

# Plot rolling standard deviation
rolling_std.plot(figsize=(20, 10), title="21 Day Rolling STD");

### Calculate and plot the correlation

In [ ]:
# Calculate and plot the correlation
df = combined_df.corr()
df.style.background_gradient(cmap="PuBu")


### Calculate and Plot Rolling 60-day Beta for Your Portfolio compared to the S&P 500

In [ ]:
# Calculate and plot Beta
covariance = combined_df["Customized"].rolling(window=60).cov(combined_df["S&P500"])
variance = combined_df["S&P500"].rolling(60).var()
beta = (covariance / variance)
beta.plot(figsize=(20, 10), title="Custom Beta");


### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Calculate Annualzied Sharpe Ratios
sharpe_ratios = (combined_df.mean() * 252) / (combined_df.std() * np.sqrt(252))
sharpe_ratios


In [ ]:
# Visualize the sharpe ratios as a bar plot
sharpe_ratios.plot(kind="bar", title="Sharpe Ratio");

### How does your portfolio do?

Write your answer here!

In [ ]:
My portfolio is the second best